# Project Proposal: Exploring Pulsar Star Data

## Introduction


#### Background
    
<p>Pulsar stars are a type of Neutron star that produces radio emission detectable on Earth, and used as probes of space-time, the interstellar medium, and states of matter<sup>1</sup>.</p> They are used to test parts of the theory of relativity, and their light emissions contain information about the physics of Neutron stars<sup>2</sup>.</p>
    


#### Research Question
    
---
<p style="text-align: center;"><b>How accurately can the Integrated Pulse Profile determine if a star is pulsar or non-pulsar?<b> </p>

---
    


#### About the Dataset

To answer the question of how effective the Integrated Pulse Profile (IPP) is at determining whether a star is pulsar, the Pulsar Dataset [HTRU2](https://www.kaggle.com/datasets/charitarth/pulsar-dataset-htru2) will be used. The data in the Pulsar Dataset was collected through the High Time Resolution Universe Survey.

<p>This dataset uses the mean, standard deviation, excess kurtosis, and skewness of the stars’ Integrated Profiles and DM-SNR curves to classify whether or not a star can be identified as a pulsar.</p>
    
><u>DM-SNR curves:</u>
>measure the radio waves released by pulsar stars once they reach Earth and have already traveled long distances in space surrounded by free electrons<sup>3</sup>.
    
><u>Integrated Pulse Profile</u>: 
>used to identify the pulsar as each profile is unique; however, pulse profiles vary slightly each period due to their signals being  non uniform and unstable. Averaging over many  thousands of rotations makes the profiles stable<sup>3</sup>.

><u> Excess Kurtosis</u>:
>means that the distribution of event outcomes have many outliers leading to fat tails on the bell shaped distribution curve<sup>4</sup>. 
    
<p>The observations under the Class variable are binary, indicating that a star is a pulsar or non-pulsar. It is for this reason that the Class variable has been converted to a factor, as the observations are discrete. The rest of the variables are continuous values<sup>1</sup>.</p>

## Exploratory Data Analysis

In [1]:
#1. Download libraries

library(tidyverse)
library(repr)
library(tidymodels)
set.seed(1)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [2]:
#2.Read in data Pulsar Star Data

pulsar_star_data<-read_csv("https://raw.githubusercontent.com/madisongill/dsci-100-2023s-group-39-section-002/main/HTRU_2.csv",col_names=FALSE)

Rows: 17898 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (9): X1, X2, X3, X4, X5, X6, X7, X8, X9

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
#3. Tidy data (add column names,condense if needed,

pulsar <- pulsar_star_data|>
rename(mean=X1,
       std_dev=X2,
       kurt=X3,
       skew=X4,
       mean_dmsnr=X5,
       std_dev_dmsnr=X6,
       kurt_dmsnr=X7,
       skew_dmsnr=X8,
       class=X9) |>
mutate(class = as_factor(class))  

In [4]:
#4.Split data into training and test set

pulsar_split<-initial_split(pulsar,prop=0.75,strata=class)
pulsar_train<-training(pulsar_split)
pulsar_test<-testing(pulsar_split)

#demonstrate data was split
train<-nrow(pulsar_train)

split<-nrow(pulsar)

percentage_train_data<-train/split*100
percentage_train_data

test<-nrow(pulsar_test)

percentage_test_data<-test/split*100
percentage_test_data

[1] 74.99721

[1] 25.00279

#### Summarizing and Visualizing the Data

The next code cell reduces the columns to be used, and summarizes their values in tables. The count and proportion of each Class, the average and standard deviation of each variable, and a count of missing values are shown.

In [12]:
reduced_train <- pulsar_train |>
    select(-(mean_dmsnr:skew_dmsnr)) #selecting for only integrated profile values

number_obs<-nrow(reduced_train)

class_count <- reduced_train |> #summarize counts of class labels
    group_by(class) |>
    summarize(count = n(),
            percent_of_train=n()/number_obs*100)#percentage of pulsar vs non pulsar

summary_train <- reduced_train |> #the average of all the predictor variables
    select(-class) |>
    map_df(mean) |>
    rename(mean_avg = mean,
           std_dev_avg = std_dev, 
           kurt_avg = kurt,
           skew_avg = skew)

means_and_counts<- as_tibble(sum(is.na(reduced_train))) |> #checking for na values
    rename(missing_vals = value) |>
    bind_cols(summary_train)  #combine summary_train and missing values check into one

standard_dev <- reduced_train |> #the standard deviation of all predictor variables, #found the sd function (5)
                    mutate(mean_sd = sd(mean),
                       std_dev_sd = sd(std_dev),
                       kurt_sd = sd(kurt),
                       skew_sd = sd(skew))|>
                        slice(1)|>
                    select(mean_sd,std_dev_sd,kurt_sd,skew_sd)

means_and_counts

standard_dev

class_count


missing_vals,mean_avg,std_dev_avg,kurt_avg,skew_avg
<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,111.1363,46.56378,0.4749385,1.757917


mean_sd,std_dev_sd,kurt_sd,skew_sd
<dbl>,<dbl>,<dbl>,<dbl>
25.51276,6.818183,1.059845,6.172478


class,count,percent_of_train
<fct>,<int>,<dbl>
0,12200,90.888773
1,1223,9.111227


The following plot is an example of using the variables to determine Class. Using information from the stars' Integrated Profiles, a relationship can be seen between the excess kurtosis and mean. It appears to be a strong negative linear relationship, with higher values of mean and lower values of excess kurtosis correlating to non-pulsar stars.

In [1]:
options(repr.plot.width = 10, repr.plot.height = 10)

pulsar_star_graph <- pulsar_train |>
ggplot(aes(x = mean, y = kurt, color = class)) +
geom_point(alpha = 0.4) +

labs(x = "Mean of the Integrated Profile", y = "Excess kurtosis of \nthe Integrated profile", color = "Class") +
scale_color_manual(labels = c("Non-pulsar", "Pulsar"),
                   values = c("orange", "steel blue")) +
ggtitle("Determining Class from Mean and Excess\n Kurtosis of Integrated Profile")+
theme(plot.title = element_text(hjust = 0.5))+
theme(text = element_text(size = 20))

pulsar_star_graph

ERROR: Error in ggplot(pulsar_train, aes(x = mean, y = kurt, color = class)): could not find function "ggplot"


## Methods

The Class will be determined using four variables. Since the IPP alone can determine whether a star is pulsar or non-pulsar, columns related to the DM-SNR curve will be excluded through selecting the mean, standard deviation, kurtosis, and skewness of the IPP.

To ensure the model can be tested, the dataset will first be split into a training and testing set. The variables will be scaled, class imbalance will be fixed. A classification model, using cross-validation, will be created using $k$ nearest neighbors to predict the star type.

The results will be visualized by plotting estimated accuracy against neighbors to show the ideal $k$ value to use. A bar graph containing true positives, false positives, true negatives, and false negatives will be added to highlight the accuracy of the model.

## Significance

The expected outcome is a highly accurate model which predicts the stars’ type. Each IPP of a star is unique, acting as a fingerprint, so the model can determine the star type while trained solely on the IPP variables.
If the model is successful, then only the IPP variables are important in identifying the stars, rather than the DM-SNR curve. This analysis can help with answering future questions such as: Which factors are effective predictors of star type? Is this relationship between predictor variables applicable to other space phenomena? Would using DM-SNR variables improve the model’s accuracy?


## References

1.https://archive.ics.uci.edu/dataset/372/htru2

2.https://www.space.com/32661-pulsars.html#:~:text=Pulsars%20have%20been%20used%20to,first%20time%20in%20February%202016


3.https://www.kaggle.com/datasets/charitarth/pulsar-dataset-htru2

4.https://www.investopedia.com/terms/e/excesskurtosis.asp#:~:text=Excess%20kurtosis%20means%20the%20distribution,by%20subtracting%20kurtosis%20by%20three.

5.https://www.digitalocean.com/community/tutorials/find-standard-deviation-in-r#:~:text=R%20is%20easy.-,R%20offers%20standard%20function%20sd(%27%20%27)%20to%20find%20the%20standard,through%20indexing%20as%20shown%20above.